# How to add new gauge-optimizations to GST results
This example demonstrates how to take a previously computed `Results` object and add new gauge-optimized version of to one of the estimates.  First, let's "pre-compute" a `ModelEstimateResults` object using `StandardGST`, which contains a single `Estimate` called "TP":

In [1]:
import pygsti
from pygsti.modelpacks import smq1Q_XYI

In [2]:
#Generate some fake data and run GST on it.
exp_design = smq1Q_XYI.get_gst_experiment_design(max_max_length=4)
mdl_datagen  = smq1Q_XYI.target_model().depolarize(op_noise=0.1, spam_noise=0.001)
ds = pygsti.construction.simulate_data(mdl_datagen, exp_design.all_circuits_needing_data, nSamples=1000, seed=1234)
data = pygsti.protocols.ProtocolData(exp_design, ds)

gst = pygsti.protocols.StandardGST("TP", gaugeopt_suite={'go0': {'itemWeights': {'gates': 1, 'spam': 1}}})
results = gst.run(data) 
results.write("example_files/regaugeopt_example")

-- Std Practice:  Iter 1 of 1  (TP) --: 
  --- Iterative MLGST: [##################################################] 100.0%  285 operation sequences ---
  Iterative MLGST Total Time: 1.7s


Next, let's load in the pre-computed results and use the `add_gauge_optimization` method of the `pygsti.objects.Estimate` object to add a new gauge-optimized version of the (gauge un-fixed) model estimate stored in `my_results.estimates['default']`.  The first argument of `add_gauge_optimization` is just a dictionary of arguments to `pygsti.gaugeopt_to_target` **except** that you don't need to specify the `Model` to gauge optimize or the target `Model` (just like the `gaugeOptParams` argument of `run_long_sequence_gst`).  The optional "`label`" argument defines the key name for the gauge-optimized `Model` and the corresponding parameter dictionary within the `Estimate`'s `.models` and `.goparameters` dictionaries, respectively.

In [3]:
my_results = pygsti.io.load_results_from_dir("example_files/regaugeopt_example", name="StandardGST")

In [4]:
estimate = my_results.estimates['TP']
estimate.add_gaugeoptimized( {'itemWeights': {'gates': 1, 'spam': 0.001}}, label="Spam 1e-3" )
mdl_gaugeopt = estimate.models['Spam 1e-3']

print(list(estimate.goparameters.keys())) # 'go0' is the default gauge-optimization label
print(mdl_gaugeopt.frobeniusdist(estimate.models['target']))

['go0', 'Spam 1e-3']
0.03923425101199379


One can also perform the gauge optimization separately and specify it using the `model` argument (this is useful when you want or need to compute the gauge optimization elsewhere):

In [5]:
mdl_unfixed = estimate.models['final iteration estimate']
mdl_gaugefixed = pygsti.gaugeopt_to_target(mdl_unfixed, estimate.models['target'], {'gates': 1, 'spam': 0.001})
estimate.add_gaugeoptimized( {'any': "dictionary", 
                              "doesn't really": "matter",
                              "but could be useful it you put gaugeopt params": 'here'},
                            model=mdl_gaugefixed, label="Spam 1e-3 custom" )
print(list(estimate.goparameters.keys()))
print(estimate.models['Spam 1e-3 custom'].frobeniusdist(estimate.models['Spam 1e-3']))

['go0', 'Spam 1e-3', 'Spam 1e-3 custom']
0.0


You can look at the gauge optimization parameters using `.goparameters`:

In [6]:
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(dict(estimate.goparameters['Spam 1e-3']))

{'_gaugeGroupEl': <pygsti.objects.gaugegroup.TPGaugeGroupElement object at 0x1227d0a58>,
 'itemWeights': {'gates': 1, 'spam': 0.001},
 'maxiter': 100,
 'model': <pygsti.objects.explicitmodel.ExplicitOpModel object at 0x122c822b0>,
 'returnAll': True,
 'targetModel': <pygsti.objects.explicitmodel.ExplicitOpModel object at 0x122caf6a0>}


Finally, note that if, in the original creation of `StandardGST`, you set **`gaugeopt_suite=None`** then no gauge optimizations are performed (there would be no "`go0`" elements) and you start with a blank slate to perform whatever gauge optimizations you want on your own.